In [32]:
import os
import numpy as np
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

In [38]:
classes = ['Alzheimer', 'Alzheimer _ 70', 'MCI' , 'MCI _ 70', 'Normal', 'Normal _ 70']

data = {}

for class_ in classes:
    dirs_ = os.listdir(class_)
    data_ = []
    for dir_ in dirs_:
        files_path = os.path.join(os.path.join(class_, dir_), '*.nii')
        for file_ in glob.glob(files_path):
            temp_file = nib.load(file_).get_data()
            if temp_file.shape[0] != 64 or temp_file.shape[1] != 64 or temp_file.shape[2] !=48:
                continue
            data_.append(temp_file)
    data[class_] = np.array(data_)

In [39]:
temp_concat = np.concatenate((data['Alzheimer'][0], data['Alzheimer'][1]), axis=3)
for i in range(2, data['Alzheimer'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer'][i]), axis=3)
data['Alzheimer'] = temp_concat
print data['Alzheimer'].shape

(64, 64, 48, 980)


In [40]:
temp_concat = np.concatenate((data['Alzheimer _ 70'][0], data['Alzheimer _ 70'][1]), axis=3)
for i in range(2, data['Alzheimer _ 70'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer _ 70'][i]), axis=3)
data['Alzheimer _ 70'] = temp_concat
print data['Alzheimer _ 70'].shape

(64, 64, 48, 1540)


In [41]:
temp_concat = np.concatenate((data['MCI'][0], data['MCI'][1]), axis=3)
for i in range(2, data['MCI'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI'][i]), axis=3)
data['MCI'] = temp_concat
print data['MCI'].shape

(64, 64, 48, 1540)


In [43]:
temp_concat = np.concatenate((data['MCI _ 70'][0], data['MCI _ 70'][1]), axis=3)
for i in range(2, data['MCI _ 70'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI _ 70'][i]), axis=3)
data['MCI _ 70'] = temp_concat
print data['MCI _ 70'].shape

(64, 64, 48, 2100)


In [44]:
temp_concat = np.concatenate((data['Normal'][0], data['Normal'][1]), axis=3)
for i in range(2, data['Normal'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal'][i]), axis=3)
data['Normal'] = temp_concat
print data['Normal'].shape

(64, 64, 48, 2380)


In [45]:
temp_concat = np.concatenate((data['Normal _ 70'][0], data['Normal _ 70'][1]), axis=3)
for i in range(2, data['Normal _ 70'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal _ 70'][i]), axis=3)
data['Normal _ 70'] = temp_concat
print data['Normal _ 70'].shape

(64, 64, 48, 1680)


In [8]:
#x = data['Alzheimer'][0]
#y = x[:,:,:,0]
#z=y[:,:,0]
#img = Image.fromarray(z.astype('uint8'))
#plt.imshow(z)
#plt.show()
#mpimg.imsave("out.tiff",z)

In [65]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
 

def build(width, height, depth, classes, weightsPath=None):
    # initialize the model
    model = Sequential()

    #first layer
    model.add(Convolution2D(20, 5, 5, border_mode="same",
        input_shape=(depth, height, width)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #second layer
    model.add(Convolution2D(50, 5, 5, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #set of FC
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    #get model summary
    model.summary()
    if weightsPath is not None:
        model.load_weights(weightsPath)
    return model
    